In [2]:
'''The following code is a solution to the agent scheduling problem. 
The problem is to assign agents to shifts in a call center. 
The goal is to minimize the total number of agents working while satisfying the following constraints:'''

from ortools.sat.python import cp_model

model = cp_model.CpModel()

num_agents = 4
num_shifts = 3
num_days = 3
all_agents = range(num_agents)
all_shifts = range(num_shifts)
all_days = range(num_days)

In [3]:
'''Assign agents to shifts
Next, we show how to assign agents to shifts subject to the following constraints:

Each shift is assigned to a single agent per day.
Each agent works at most one shift per day.
Here's the code that creates the first condition.'''

shifts = {}
for n in all_agents:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d,
                    s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

In [4]:
# For each shift, the sum of the agents assigned to that shift is 1.

for d in all_days:
    for s in all_shifts:
        model.AddExactlyOne(shifts[(n, d, s)] for n in all_agents)

In [5]:
# The code that requires that each agent works at most one shift per day.

for n in all_agents:
    for d in all_days:
        model.AddAtMostOne(shifts[(n, d, s)] for s in all_shifts)

In [6]:
'''Assign shifts evenly
Next, we show how to assign shifts to agents as evenly as possible. Since there are nine shifts over the three-day period, we can assign two shifts to each of the four agents. After that there will be one shift left over, which can be assigned to any agent.

The following code ensures that each agent works at least two shifts in the three-day period.'''

# Try to distribute the shifts evenly, so that each agent works
# min_shifts_per_agent shifts. If this is not possible, because the total
# number of shifts is not divisible by the number of agents, some agents will
# be assigned one more shift.

min_shifts_per_agent = (num_shifts * num_days) // num_agents
if num_shifts * num_days % num_agents == 0:
    max_shifts_per_agent = min_shifts_per_agent
else:
    max_shifts_per_agent = min_shifts_per_agent + 1
for n in all_agents:
    num_shifts_worked = []
    for d in all_days:
        for s in all_shifts:
            num_shifts_worked.append(shifts[(n, d, s)])
    model.Add(min_shifts_per_agent <= sum(num_shifts_worked))
    model.Add(sum(num_shifts_worked) <= max_shifts_per_agent)

In [7]:
'''For the given values of num_agents = 4, num_shifts = 3, and num_days = 3, 
the expression min_shifts_per_agent has the value (3 * 3 // 4) = 2, 
so you can assign at least two shifts to each agent. This is guaranteed by the constraint.'''

model.Add(min_shifts_per_agent <= sum(num_shifts_worked))

In [8]:
'''Since there are nine total shifts over the three-day period, 
there is one remaining shift after assigning two shifts to each agent. 
The extra shift can be assigned to any agent.'''

model.Add(sum(num_shifts_worked) <= max_shifts_per_agent)

In [9]:
# In a non-optimization model, you can enable the search for all solutions.

solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

In [10]:
# You need to register a callback on the solver that will be called at each solution.

class agentsPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_agents, num_days, num_shifts, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_agents = num_agents
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print('Solution %i' % self._solution_count)
        for d in range(self._num_days):
            print('Day %i' % d)
            for n in range(self._num_agents):
                is_working = False
                for s in range(self._num_shifts):
                    if self.Value(self._shifts[(n, d, s)]):
                        is_working = True
                        print('  agent %i works shift %i' % (n, s))
                if not is_working:
                    print('  agent {} does not work'.format(n))
        if self._solution_count >= self._solution_limit:
            print('Stop search after %i solutions' % self._solution_limit)
            self.StopSearch()

    def solution_count(self):
        return self._solution_count

# Display the first five solutions.
solution_limit = 5
solution_printer = agentsPartialSolutionPrinter(shifts, num_agents,
                                                num_days, num_shifts,
                                                solution_limit)

In [11]:
# The following code calls the solver and displays the first five solutions.

status = solver.SearchForAllSolutions(model, solution_printer)

Solution 1
Day 0
  agent 0 does not work
  agent 1 works shift 0
  agent 2 works shift 1
  agent 3 works shift 2
Day 1
  agent 0 works shift 2
  agent 1 does not work
  agent 2 works shift 1
  agent 3 works shift 0
Day 2
  agent 0 works shift 2
  agent 1 works shift 1
  agent 2 works shift 0
  agent 3 does not work
Solution 2
Day 0
  agent 0 works shift 0
  agent 1 does not work
  agent 2 works shift 1
  agent 3 works shift 2
Day 1
  agent 0 does not work
  agent 1 works shift 2
  agent 2 works shift 1
  agent 3 works shift 0
Day 2
  agent 0 works shift 2
  agent 1 works shift 1
  agent 2 works shift 0
  agent 3 does not work
Solution 3
Day 0
  agent 0 works shift 0
  agent 1 does not work
  agent 2 works shift 1
  agent 3 works shift 2
Day 1
  agent 0 works shift 1
  agent 1 works shift 2
  agent 2 does not work
  agent 3 works shift 0
Day 2
  agent 0 works shift 2
  agent 1 works shift 1
  agent 2 works shift 0
  agent 3 does not work
Solution 4
Day 0
  agent 0 works shift 0
  agent 

In [62]:
# importing google or tools and declaring model template
from ortools.sat.python import cp_model
model = cp_model.CpModel()
# declare empty list that will be used for storing indices for agent-shift-day combination
shiftoptions = {}

# set number of agents, days and schedules as well as max schedules per day, 
# as well as max shift amount difference per agent
agents = 5
shifts = 3
days = 7
maxshiftsperday = 1
maxdifference = 1

# create a tuple as a shift option list index, for each combination of agent, shift and day
# use google or tools to create a boolean variable indicating if given agent works on that day, in that shift
for x in range(agents):
    for y in range(days):
        for z in range(shifts):
            shiftoptions[(x,y,z)] = model.NewBoolVar("shift with id" + str(x) + " " + str(y) + " " + str(z))

# now we add the constraint of shift only being assigned to one agent
for y in range(days):
    for z in range(shifts):
        model.Add(sum(shiftoptions[(x, y, z)] for x in range(agents)) == 1)
# now we add the constraint of a agent only working one shift per day
for x in range(agents):
    for y in range(days):
        model.Add(sum(shiftoptions[(x,y,z)] for z in range(shifts)) <= 1)
# now we add the constraint of all agents having the same amount of shifts, with some deviations allowed for with a maximum allowed difference
minshiftsperagent = (shifts * days) // agents
print(minshiftsperagent)
maxshiftsperagent = minshiftsperagent + maxdifference
for x in range(agents):
    shiftsassigned = 0
    for y in range(days):
        for z in range(shifts):
            shiftsassigned += shiftoptions[(x,y,z)]
    model.Add(minshiftsperagent <= shiftsassigned)
    model.Add(shiftsassigned <= maxshiftsperagent)

# before solving the problem I add a solution printer (this code is taken directly from Google's documentation)
class SolutionPrinterClass(cp_model.CpSolverSolutionCallback):
    def __init__(self, shiftoptions, agents, days, shifts, sols):
        val = cp_model.CpSolverSolutionCallback.__init__(self)
        self._shiftoptions = shiftoptions
        self._agents = agents
        self._days = days
        self._shifts = shifts
        self._solutions = set(sols)
        self._solution_count = 0
    def on_solution_callback(self):
        if self._solution_count in self._solutions:
            print("solution " + str(self._solution_count))
            for y in range(self._days):
                print("day " + str(y))
                for x in range(self._agents):
                    is_working = False
                    for z in range(self._shifts):
                        if self.Value(self._shiftoptions[(x,y,z)]):
                            is_working = True
                            print("agent " +str(x) +" works day " + str(y) +" shift " + str(z))
                    if not is_working:
                        print('  agent {} does not work'.format(x))
            print()
        self._solution_count += 1
    def solution_count(self):
        return self._solution_count

# solve the model
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# solve it and check if solution was feasible
solutionrange = range(1) # we want to display 1 feasible results (the first one in the feasible set)
solution_printer = SolutionPrinterClass(shiftoptions, agents,
                                        days, shifts, solutionrange)
solver.Solve(model, solution_printer)

4
solution 0
day 0
  agent 0 does not work
  agent 1 does not work
agent 2 works day 0 shift 1
agent 3 works day 0 shift 2
agent 4 works day 0 shift 0
day 1
agent 0 works day 1 shift 1
  agent 1 does not work
agent 2 works day 1 shift 2
agent 3 works day 1 shift 0
  agent 4 does not work
day 2
agent 0 works day 2 shift 2
agent 1 works day 2 shift 1
agent 2 works day 2 shift 0
  agent 3 does not work
  agent 4 does not work
day 3
agent 0 works day 3 shift 2
agent 1 works day 3 shift 1
agent 2 works day 3 shift 0
  agent 3 does not work
  agent 4 does not work
day 4
agent 0 works day 4 shift 0
agent 1 works day 4 shift 1
  agent 2 does not work
  agent 3 does not work
agent 4 works day 4 shift 2
day 5
  agent 0 does not work
  agent 1 does not work
agent 2 works day 5 shift 0
agent 3 works day 5 shift 2
agent 4 works day 5 shift 1
day 6
  agent 0 does not work
agent 1 works day 6 shift 0
  agent 2 does not work
agent 3 works day 6 shift 1
agent 4 works day 6 shift 2



4

In [63]:
'''Now we are doing the same thing, but we are adding working hours per shift and we minimize the total number of agents working.'''

from ortools.sat.python import cp_model
from collections import Counter
import numpy as np
import pprint

# This program tries to find an optimal assignment of agents to shifts
# (4 shifts per day, for 5 days), subject to some constraints (see below).
# We add the demand of agents in each of the 13 houtly time slots.
# Each agent can work at most one shift per day.
# The objective is to minimize the maximum number of agents working in a day.
# There are 4 shifts available per day, and 5 days in the week.
# There are 8 agents in the call center available for the week.

num_agents = 8
num_shifts = 4
num_days = 5
num_hours = 13
all_agents = range(num_agents)
all_shifts = range(num_shifts)
all_days = range(num_days)
all_hours = range(num_hours)

# This is a dictionary that contains the name of the shift and an array of 13 hours where 1 means that the agent is working in that hour.
hours_shift = {"full_am": [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
            "full_ams": [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
            "full_pm": [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
            "full_pms": [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
}

# We extract the hours from the dictionary and we create a list of shifts.
shifts_list = list(hours_shift.values())

# This is the list of the demand of agents in each of the 13 hours. There are five lists, one for each day.
demand = [[1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1]]

# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.

shifts = np.empty(shape=(num_agents, num_days, num_shifts), dtype='object')

for a in all_agents:
    for d in all_days:
        for s in all_shifts:
            shifts[a, d, s] = model.NewBoolVar('shift_a%id%is%i' % (a, d, s))

# Each shift is assigned to exactly one agent in a week.

for d in all_days:
    for s in all_shifts:
        model.Add(sum(shifts[a, d, s] for a in all_agents) == 1)

# Each agent cannot work more than 45 hours in a week.

for a in all_agents:
    model.Add(sum(shifts[(a, d, s)] * sum(shifts_list[s]) for d in all_days for s in all_shifts) <= 45)

# for a in all_agents:
#     model.Add(sum(sum(shifts[a, d, s] for s in all_shifts) for d in all_days) <= 45)

# Each agent works at most one shift per day.

for a in all_agents:
    for d in all_days:
        model.Add(sum(shifts[a, d, s] for s in all_shifts) <= 1)

# The number of agents available in an hour has to be equal or greater than the demand.

for d in all_days:
    for h in range(13):
        model.Add(sum(shifts[a, d, s] * shifts_list[s][h] for a in all_agents for s in all_shifts) >= demand[d][h])

# The objective is to minimize the maximum number of agents working in a day.

obj_var = model.NewIntVar(0, num_agents, 'max_agents')

for d in all_days:
    model.Add(obj_var >= sum(shifts[a, d, s] for a in all_agents for s in all_shifts))

model.Minimize(obj_var)

# Creates the solver and solve.

solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print solution.

if status == cp_model.OPTIMAL:
    print('Maximum of agents working in a day = %i' % solver.ObjectiveValue())
    print()
    for d in all_days:
        print('Day', d)
        for a in all_agents:
            for s in all_shifts:
                if solver.BooleanValue(shifts[a, d, s]):
                    print('Agent', a, 'works shift', s, '(hours', shifts_list[s], ')')
        print()

Maximum of agents working in a day = 4

Day 0
Agent 0 works shift 1 (hours [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0] )
Agent 1 works shift 3 (hours [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0] )
Agent 2 works shift 0 (hours [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0] )
Agent 7 works shift 2 (hours [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1] )

Day 1
Agent 0 works shift 2 (hours [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1] )
Agent 1 works shift 1 (hours [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0] )
Agent 2 works shift 0 (hours [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0] )
Agent 7 works shift 3 (hours [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0] )

Day 2
Agent 0 works shift 2 (hours [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1] )
Agent 3 works shift 0 (hours [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0] )
Agent 5 works shift 3 (hours [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0] )
Agent 7 works shift 1 (hours [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0] )

Day 3
Agent 0 works shift 1 (hours [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0] )
Agent 1 wo

In [83]:
'''Now we are doing the same thing, but we are adding working hours per shift and we minimize the total number of agents working.
Additionally  we are adding three types of contracts: full time, part time and flexible. The full time contract is 45 hours per week, 
the part time contract is 30 hours per week and the flexible contract is 20 hours per week. Each week has now 7 days.
Preferrable the flexible contract is used for the last 2 days of the week, the objective is to minimize the maximum number of agents working in a day.'''

from ortools.sat.python import cp_model
from collections import Counter
import numpy as np
import pprint

# This program tries to find an optimal assignment of agents to shifts
# 4 shifts per contract for the 7 days, each agent has a contract, subject to some constraints (see below).
# We add the demand of agents in each of the 13 houtly time slots.
# Each agent can work at most one shift per day.
# The objective is to minimize the maximum number of agents working in a day.
# There are 10 agents in the call center available for the week.

num_agents = 10
num_shifts = 4
num_days = 7
# There are 3 types of contracts: full time, part time and flexible.
num_contracts = 3
# Number of hours per day.
num_hours = 13

all_agents = range(num_agents)
all_shifts = range(num_shifts)
all_days = range(num_days)
all_contracts = range(num_contracts)
all_hours = range(num_hours)

# This is a dictionary for the full time contract that contains the name of the shift and an array of 13 hours where 1 means that the agent is working in that hour.
full_contract_shift = {"full_am": [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
                       "full_ams": [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
                       "full_pm": [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                       "full_pms": [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]}

# This is a dictionary for the part time contract that contains the name of the shift and an array of 13 hours where 1 means that the agent is working in that hour.
part_contract_shift = {"part_am": [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                       "part_ams": [0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                       "part_pm": [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
                       "part_pms": [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0]}

# This is a dictionary for the flexible contract that contains the name of the shift and an array of 13 hours where 1 means that the agent is working in that hour.
flex_contract_shift = {"flex_am": [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                       "flex_ams": [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                       "flex_pm": [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
                       "flex_pms": [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0]}

# We extract the hours from the dictionary for the fulltime contract and we create a list of shifts.
shifts_full_contract= list(full_contract_shift.values())

# We extract the hours from the dictionary for the parttime contract and we create a list of shifts.
shifts_part_contract= list(part_contract_shift.values())

# We extract the hours from the dictionary for the flexible contract and we create a list of shifts.
shifts_flex_contract= list(flex_contract_shift.values())

# We create a list of all the shifts for all the contracts. We have 12 shifts. The first 4 are for the fulltime contract, the next 4 are for the parttime contract and the last 4 are for the flexible contract.
list_shift_each_contract = [shifts_full_contract, shifts_part_contract, shifts_flex_contract]

# This is the list of the demand of agents in each of the 13 hours. There are seven lists, one for each day.
demand = [[1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1]]

In [91]:
# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
# shifts[(a, d, c, s)]: agent 'a' has a contract 'c' and works shift 's' on day 'd'.
shifts = {}
for a in all_agents:
    for d in all_days:
        for c in all_contracts:
            for s in all_shifts:
                shifts[(a, d, c, s)] = model.NewBoolVar('shift_a%id%ic%is%i' % (a, d, c, s))

In [92]:
# Each agent is assigned to at most one contract during the week.
for a in all_agents:
    for d in all_days:
        model.Add(sum(shifts[(a, d, c, s)] for c in all_contracts for s in all_shifts) <= 1)

In [93]:
# Each agent is assigned to at most one shift per day within his contract.
for a in all_agents:
    for d in all_days:
        for c in all_contracts:
            model.Add(sum(shifts[(a, d, c, s)] for s in all_shifts) <= 1)

In [94]:
# The number of agents available in a given hour has to be equal or greater than the demand.
for d in all_days:
    for h in all_hours:
        model.Add(sum(shifts[(a, d, c, s)] * list_shift_each_contract[c][s][h] for a in all_agents for c in all_contracts for s in all_shifts) >= demand[d][h])

In [95]:
# The objective is to minimize the number of agents working during the week.
obj_var = model.NewIntVar(0, num_agents, 'num_agents_working')
model.AddMaxEquality(obj_var, [sum(shifts[(a, d, c, s)] for a in all_agents for c in all_contracts for s in all_shifts) for d in all_days])
model.Minimize(obj_var)

In [96]:
# Creates the solver and solve.
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print solution.
if status == cp_model.OPTIMAL:
    print('Total agents working = %i' % solver.ObjectiveValue())
    for d in all_days:
        print('Day', d)
        # for h in all_hours:
        for a in all_agents:
            for c in all_contracts:
                for s in all_shifts:
                    if solver.Value(shifts[(a, d, c, s)]) == 1:
                        # if list_shift_each_contract[c][s][h] == 1:
                        print('Agent', a, 'works shift', s, 'of contract', c, 'on day', d)

In [42]:
# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
# shifts[(a, d, c, s)]: agent 'a' has a contract 'c' and works shift 's' on day 'd'.
shifts = {}
for a in all_agents:
    for d in all_days:
        for c in all_contracts:
            for s in all_shifts:
                shifts[(a, d, c, s)] = model.NewBoolVar('shift_a%id%ic%is%i' % (a, d, c, s))

# Each agent is assigned to one contract during the week.
for a in all_agents:
    for d in all_days:
        model.Add(sum(shifts[(a, d, c, s)] for c in all_contracts for s in all_shifts) == 1)

# Each agent is assigned to one shift from its contract per week.
for a in all_agents:
    for c in all_contracts:
        model.Add(sum(shifts[(a, d, c, s)] for d in all_days for s in all_shifts) == 1)

# Each agent works no more than one shift per day.
for a in all_agents:
    for d in all_days:
        model.Add(sum(shifts[(a, d, c, s)] for c in all_contracts for s in all_shifts) <= 1)

# Each agent assigned to full time contract cannot work more than 45 hours in a week.
# We need to sum each hour of the week for each agent and each shift.
for a in all_agents:
    for s in all_shifts:
        model.Add(sum(shifts[(a, d, 0, s)] * list_shift_each_contract[0][s][h] for d in all_days for h in all_hours) <= 45)


# Each agent assigned to part time contract cannot work more than 30 hours in a week.
# We need to sum each hour of the week for each agent and each shift.
for a in all_agents:
    for s in all_shifts:
        model.Add(sum(shifts[(a, d, 1, s)] * list_shift_each_contract[1][s][h] for d in all_days for h in all_hours) <= 30)

# Each agent assigned to flexible contract cannot work more than 20 hours in a week.
# We need to sum each hour of the week for each agent and each shift.
for a in all_agents:
    for s in all_shifts:
        model.Add(sum(shifts[(a, d, 2, s)] * list_shift_each_contract[2][s][h] for d in all_days for h in all_hours) <= 20)

# The number of agents available in a given hour has to be equal or greater than the demand.
for d in all_days:
    for h in all_hours:
        model.Add(sum(shifts[(a, d, c, s)] * list_shift_each_contract[c][s][h] for a in all_agents for c in all_contracts for s in all_shifts) >= demand[d][h])

# The objective is to minimize the number of agents assigned to full time contract.
# model.Minimize(sum(shifts[(a, d, 0, s)] for a in all_agents for d in all_days for s in all_shifts))

# Creates the solver and solve.
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print solution for each day.
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Total cost = %i' % solver.ObjectiveValue())
    print()
    for d in all_days:
        print('Day', d)
        for a in all_agents:
            for c in all_contracts:
                for s in all_shifts:
                    if solver.BooleanValue(shifts[(a, d, c, s)]):
                        print('Agent', a, 'works contract', c, 'shift', s, full_contract_shift[s])
        print()


KeyError: (0, 0, 1, 0)

In [31]:
solver.Solve(model)

3

In [59]:
'''Now we are doing the same thing, but we are adding working hours per shift and we minimize the total number of agents working.
Additionally  we are adding three types of contracts: full time, part time and flexible. The full time contract is 45 hours per week, 
the part time contract is 30 hours per week and the flexible contract is 20 hours per week. Each week has now 7 days.
Preferrable the flexible contract is used for the last 2 days of the week, the objective is to minimize the maximum number of agents working in a day.'''

from ortools.sat.python import cp_model
from collections import Counter
import numpy as np
import pprint

# This program tries to find an optimal assignment of agents to shifts
# 4 shifts per contract for the 7 days, each agent has a contract, subject to some constraints (see below).
# We add the demand of agents in each of the 13 houtly time slots.
# Each agent can work at most one shift per day.
# The objective is to minimize the maximum number of agents working in a day.
# There are 10 agents in the call center available for the week.

num_agents = 10
num_shifts = 4
num_days = 7
# There are 3 types of contracts: full time, part time and flexible.
num_contracts = 1
# Number of hours per day.
num_hours = 13

all_agents = range(num_agents)
all_shifts = range(num_shifts)
all_days = range(num_days)
all_contracts = range(num_contracts)
all_hours = range(num_hours)

# This is a dictionary for the full time contract that contains the name of the shift and an array of 13 hours where 1 means that the agent is working in that hour.
full_contract_shift = {"full_am": [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
                       "full_ams": [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
                       "full_pm": [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                       "full_pms": [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]}

# # This is a dictionary for the part time contract that contains the name of the shift and an array of 13 hours where 1 means that the agent is working in that hour.
# part_contract_shift = {"part_am": [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
#                        "part_ams": [0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
#                        "part_pm": [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
#                        "part_pms": [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0]}

# # This is a dictionary for the flexible contract that contains the name of the shift and an array of 13 hours where 1 means that the agent is working in that hour.
# flex_contract_shift = {"flex_am": [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#                        "flex_ams": [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
#                        "flex_pm": [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
#                        "flex_pms": [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0]}

# We extract the hours from the dictionary for the fulltime contract and we create a list of shifts.
shifts_full_contract= list(full_contract_shift.values())

# # We extract the hours from the dictionary for the parttime contract and we create a list of shifts.
# shifts_part_contract= list(part_contract_shift.values())

# # We extract the hours from the dictionary for the flexible contract and we create a list of shifts.
# shifts_flex_contract= list(flex_contract_shift.values())

# We create a list of all the shifts for all the contracts. We have 12 shifts. The first 4 are for the fulltime contract, the next 4 are for the parttime contract and the last 4 are for the flexible contract.
list_shift_each_contract = [shifts_full_contract]#, shifts_part_contract, shifts_flex_contract]

# This is the list of the demand of agents in each of the 13 hours. There are seven lists, one for each day.
demand = [[1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1],
            [1, 2, 2, 3, 3, 1, 3, 3, 3, 2, 2, 1, 1]]

# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
# shifts[(a, d, c, s)]: agent 'a' has a contract 'c' and works shift 's' on day 'd'.
shifts = {}
for a in all_agents:
    for d in all_days:
        for c in all_contracts:
            for s in all_shifts:
                shifts[(a, d, c, s)] = model.NewBoolVar('shift_a%id%ic%is%i' % (a, d, c, s))

# Each agent is assigned to one contract during the week.
for a in all_agents:
    for d in all_days:
        model.Add(sum(shifts[(a, d, c, s)] for c in all_contracts for s in all_shifts) == 1)

# Each agent is assigned to one shift from its contract per week.
for a in all_agents:
    for c in all_contracts:
        model.Add(sum(shifts[(a, d, c, s)] for d in all_days for s in all_shifts) == 1)

# Each agent works no more than one shift per day.
for a in all_agents:
    for d in all_days:
        model.Add(sum(shifts[(a, d, c, s)] for c in all_contracts for s in all_shifts) <= 1)

# Each agent assigned to full time contract cannot work more than 45 hours in a week.
for a in all_agents:
    model.Add(sum(shifts[(a, d, 0, s)] * sum(list_shift_each_contract[0][s]) for d in all_days for s in all_shifts) <= 45)

# # Each agent assigned to part time contract cannot work more than 30 hours in a week.
# for a in all_agents:
#     model.Add(sum(shifts[(a, d, 1, s)] * sum(list_shift_each_contract[1][s]) for d in all_days for s in all_shifts) <= 30)

# # Each agent assigned to flexible contract cannot work more than 20 hours in a week.
# for a in all_agents:
#     model.Add(sum(shifts[(a, d, 2, s)] * sum(list_shift_each_contract[2][s]) for d in all_days for s in all_shifts) <= 20)

# The number of agents available in a given hour has to be equal or greater than the demand.
for d in all_days:
    for h in all_hours:
        model.Add(sum(shifts[(a, d, c, s)] * list_shift_each_contract[c][s][h] for a in all_agents for c in all_contracts for s in all_shifts) >= demand[d][h])

# The objective is to minimize the maximum number of agents working in a week.
obj_var = model.NewIntVar(0, num_agents, 'max_agents')

for d in all_days:
    model.AddMaxEquality(obj_var, [sum(shifts[(a, d, c, s)] for a in all_agents for c in all_contracts for s in all_shifts)])

model.Minimize(obj_var)

# Creates the solver and solve.
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print solution for each day.
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Total cost = %i' % solver.ObjectiveValue())
    print()
    for d in all_days:
        print('Day', d)
        for a in all_agents:
            for c in all_contracts:
                for s in all_shifts:
                    if solver.BooleanValue(shifts[(a, d, c, s)]):
                        print('Agent', a, 'works contract', c, 'shift', s, full_contract_shift[s])
        print()


In [60]:
solver.Solve(model)

3